In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import paramiko
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Transfer_data_to_64_saatva_for_David'

In [2]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root, file)

In [3]:
host = "64.237.51.251" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "jian@juba2017" #hard-coded
username = "jian" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)

# Traffic_Files

In [8]:
traffic_files_2018=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
traffic_files_2018=[x for x in traffic_files_2018 if "TrafficWeekly" in x]

traffic_files_2019=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
traffic_files_2019=[x for x in traffic_files_2019 if "TrafficWeekly" in x]

traffic_files=sorted(traffic_files_2018+traffic_files_2019)
len(traffic_files)

79

In [11]:
remote_folder="/mnt/drv5/saatva/Big_Lots_Data/weekly_traffics/"
    
for local_file in traffic_files:
    df=pd.read_table(local_file,nrows=1,dtype=str,sep="|")
    week_end_dt=df['week_end_dt'].tolist()[0]
    basename="MediaStormTrafficWeekly_"+str(week_end_dt)+".txt"
    remote_path=remote_folder+basename
    sftp.put(local_file,remote_path)

# Weekly Sales Files

In [10]:
weekly_files_2018=list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/"))
weekly_files_2018=[x for x in weekly_files_2018 if "SalesWeekly" in x]

weekly_files_2019=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
weekly_files_2019=[x for x in weekly_files_2019 if "SalesWeekly" in x]

weekly_files=weekly_files_2018+weekly_files_2019
len(weekly_files)

73

In [11]:
remote_folder="/mnt/drv5/saatva/Big_Lots_Data/weekly_sales/"
    
for local_file in weekly_files:
    df=pd.read_table(local_file,nrows=1,dtype=str,sep="|")
    week_end_dt=df['week_end_dt'].tolist()[0]
    basename="MediaStormSalesWeekly_"+str(week_end_dt)+".txt"
    remote_path=remote_folder+basename
    sftp.put(local_file,remote_path)

# Daily Sales Item

In [19]:
daily_files_item_hist=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"))
daily_files_item_hist=[x for x in daily_files_item_hist if ".txt" in x]
df_daily_files_item_hist=pd.DataFrame({"file_path":daily_files_item_hist})
df_daily_files_item_hist['week_end_dt']=df_daily_files_item_hist['file_path'].apply(lambda x: x.split("MediaStormDailySalesHistory")[1][:8])

item_min_date=min(df_daily_files_item_hist['week_end_dt'])
item_max_date=max(df_daily_files_item_hist['week_end_dt'])

# Wrong folder below for 2019, resent at the bottom
'''
daily_files_item_2019=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_files_item_2019=[x for x in daily_files_item_2019 if "aily" in x]
df_daily_files_item_2019=pd.DataFrame({"file_path":daily_files_item_2019})
df_daily_files_item_2019['week_end_dt']=df_daily_files_item_2019['file_path'].apply(lambda x: x.split("_by_weeks/MediaStorm_")[1][:10].replace("-",""))
'''

df_daily_files=df_daily_files_item_hist.append(df_daily_files_item_2019)
df_daily_files=df_daily_files.reset_index()
del df_daily_files['index']

In [16]:
remote_folder="/mnt/drv5/saatva/Big_Lots_Data/daily_sales/item_level/"


for index,row in df_daily_files.iterrows():
    local_file=row['file_path']
    week_end_dt=row['week_end_dt']
    basename="MediaStormDailySales_"+str(week_end_dt)+".txt"
    remote_path=remote_folder+basename
    sftp.put(local_file,remote_path)
    

Socket exception: Connection reset by peer (104)


# Daily Sales Subclass

In [21]:
daily_files_subclass_hist=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data_subclasslevel/"))
daily_files_subclass_hist=[x for x in daily_files_subclass_hist if ".txt" in x]
df_daily_files_subclass_hist=pd.DataFrame({"file_path":daily_files_subclass_hist})
df_daily_files_subclass_hist['week_end_dt']=df_daily_files_subclass_hist['file_path'].apply(lambda x: x.split("MediaStormDailySales_week_ending_")[1][:10].replace("-",""))
df_daily_files_subclass_hist=df_daily_files_subclass_hist[df_daily_files_subclass_hist['week_end_dt']<item_min_date]

df_daily_files_subclass_hist=df_daily_files_subclass_hist.reset_index()
del df_daily_files_subclass_hist['index']

df_daily_files_subclass_hist.shape

(102, 2)

In [18]:
remote_folder="/mnt/drv5/saatva/Big_Lots_Data/daily_sales/subclass_level/"

for index,row in df_daily_files_subclass_hist.iterrows():
    local_file=row['file_path']
    week_end_dt=row['week_end_dt']
    basename="MediaStormDailySales_"+str(week_end_dt)+".txt"
    remote_path=remote_folder+basename
    sftp.put(local_file,remote_path)

In [7]:
# Fill the gap from 2018-6-16 to 2018-8-11
# checked as all Subclass level
# All 1 week
path_daily_20180616="/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-16/MediaStormDailySales.txt"
path_daily_20180623="/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-23/MediaStormDailySales.txt"
path_daily_20180630="/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-06-30/MediaStormDailySales.txt"
path_daily_20180707="/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-07/MediaStormDailySales.txt"
path_daily_20180714="/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-14/MediaStormDailySales20180717-113630-767.txt"
path_daily_20180721="/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-21/MediaStormDailySales20180724-113327-741.txt"
path_daily_20180728="/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-07-28/MediaStormDailySales20180731-111804-489.txt"
path_daily_20180804="/home/jian/BigLots/2018_by_weeks/MediaStorm_2018-08-04/MediaStormDailySales20180807-111637-702.txt"

all_gap_subclass=[path_daily_20180616,path_daily_20180623,path_daily_20180630,path_daily_20180707,path_daily_20180714,
                 path_daily_20180721,path_daily_20180728,path_daily_20180804]


remote_folder="/mnt/drv5/saatva/Big_Lots_Data/daily_sales/subclass_level/"

for local_path in all_gap_subclass:
    
    week_end_dt=local_path.split("y_weeks/MediaStorm_")[1][:10].replace("-","")
    basename="MediaStormDailySales_"+str(week_end_dt)+".txt"
    remote_path=remote_folder+basename
    sftp.put(local_path,remote_path)

# Daily Sales Combined Old

In [27]:
combined_up_to_20180922='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
remote_folder="/mnt/drv5/saatva/Big_Lots_Data/daily_sales/"
basename=os.path.basename(combined_up_to_20180922)
remote_path=remote_folder+basename
sftp.put(combined_up_to_20180922,remote_path)


<SFTPAttributes: [ size=16405838438 uid=518 gid=558 mode=0o100644 atime=1559511194 mtime=1559581022 ]>

In [28]:
combined_up_to_20180922='/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q1/crm_newscore_0922/combinedtransactions_0922.csv'
test_df=pd.read_table(combined_up_to_20180922,dtype=str,nrows=10,sep=",")
del test_df['merch_cat']
test_df=test_df.drop_duplicates()

Socket exception: Connection reset by peer (104)


# Sign Up Master Files

In [10]:
remote_folder="/mnt/drv5/saatva/Big_Lots_Data/master_files/"

combined_master='/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/combined_masterids_up_to_20181229_JL.csv'
lapsed_master='/home/jian/Projects/Big_Lots/Loyal_members/loyalty_register_data/MediaStorm_Lapsed_Reward_Member_Master_from2014-08-26to2017-02-26.zip'
master_2018Q1_to_2019Q1="/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip"

list_cumulative_old_master=[combined_master,lapsed_master,master_2018Q1_to_2019Q1]
for local_path in list_cumulative_old_master:
    basename=os.path.basename(local_path)
    remote_path=remote_folder+basename
    sftp.put(local_path,remote_path)

In [13]:
weekly_master_files_2019Q2_plus=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))

weekly_master_files_2019Q2_plus=[x for x in weekly_master_files_2019Q2_plus if "aster" in x]

df_weekly_master_files_2019Q2_plus=pd.DataFrame({"file_path":weekly_master_files_2019Q2_plus})
df_weekly_master_files_2019Q2_plus['week_end_dt']=df_weekly_master_files_2019Q2_plus['file_path'].apply(lambda x: x.split("by_weeks/MediaStorm_")[1][:10].replace(",",""))
df_weekly_master_files_2019Q2_plus=df_weekly_master_files_2019Q2_plus[df_weekly_master_files_2019Q2_plus['week_end_dt']>"2019-05-04"]

df_weekly_master_files_2019Q2_plus.shape

,file_path,week_end_dt
16,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-11
17,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-18
18,/home/jian/BigLots/2019_by_weeks/MediaStorm_20...,2019-05-25


In [17]:
remote_folder="/mnt/drv5/saatva/Big_Lots_Data/master_files/master_weekly/"

for index,row in df_weekly_master_files_2019Q2_plus.iterrows():
    local_file=row['file_path']
    week_end_dt=row['week_end_dt']
    basename="MediaStormMasterWeekly_"+str(week_end_dt)+".txt"
    remote_path=remote_folder+basename
    sftp.put(local_file,remote_path)

In [18]:
sftp.close()

In [7]:
daily_files_item_2019=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/"))
daily_files_item_2019=[x for x in daily_files_item_2019 if "aily" in x]
daily_files_item_2019=[x for x in daily_files_item_2019 if "2019" in x]
df_daily_files_item_2019=pd.DataFrame({"file_path":daily_files_item_2019})
df_daily_files_item_2019['week_end_dt']=df_daily_files_item_2019['file_path'].apply(lambda x: x.split("StormDailySalesHistory")[1][:8])
df_daily_files_item_2019['week_end_dt']=df_daily_files_item_2019['week_end_dt'].astype(int)
df_daily_files_item_2019=df_daily_files_item_2019.reset_index()
del df_daily_files_item_2019['index']

In [8]:
df_daily_files_item_2019

,file_path,week_end_dt
0,/home/jian/BigLots/hist_daily_data_itemlevel_d...,20190105
1,/home/jian/BigLots/hist_daily_data_itemlevel_d...,20190112
2,/home/jian/BigLots/hist_daily_data_itemlevel_d...,20190119
3,/home/jian/BigLots/hist_daily_data_itemlevel_d...,20190126
4,/home/jian/BigLots/hist_daily_data_itemlevel_d...,20190202
5,/home/jian/BigLots/hist_daily_data_itemlevel_d...,20190209


In [9]:
remote_folder="/mnt/drv5/saatva/Big_Lots_Data/daily_sales/item_level/"

for index,row in df_daily_files_item_2019.iterrows():
    local_file=row['file_path']
    week_end_dt=row['week_end_dt']
    basename="MediaStormDailySales_"+str(week_end_dt)+".txt"
    remote_path=remote_folder+basename
    sftp.put(local_file,remote_path)

In [10]:
sftp.close()